about the dataset:
1. id:unique id for a news article
2. title: the title of a news article
3. author: author of the news article
4. text: the text of the article, could be incomplete
5. label : a label that marks whether the news article is real or fake:

    1: Fake news
    0: real news
    

Importing the Depedencies

In [202]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [203]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [204]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [206]:
#loading the dataset to a panadas DataFrame
news_dataset = pd.read_csv('/content/dataset.csv')

In [207]:
news_dataset.shape

(18822, 5)

In [208]:
#print the first 5 rows of the dataframe
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1.0
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0.0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1.0
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1.0
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1.0


In [209]:
#couting the number of missing values in the dataset
news_dataset.isnull().sum()

id           0
title      496
author    1784
text        37
label        1
dtype: int64

In [210]:
#replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [211]:
# merging the author name and news title
news_dataset['content'] = news_dataset['author'] + ' ' + ['title']


In [212]:
print(news_dataset['content'])

0              Darrell Lucus title
1            Daniel J. Flynn title
2         Consortiumnews.com title
3            Jessica Purkiss title
4             Howard Portnoy title
                   ...            
18817    Breitbart Jerusalem title
18818                        title
18819            Liz Robbins title
18820                        title
18821                        title
Name: content, Length: 18822, dtype: object


In [213]:
#separating the data & label
X = news_dataset.drop(columns = 'label', axis=1)
Y = news_dataset['label']

In [214]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
18817  18817  Steven Tyler Lands in Israel Ahead of Aerosmit...   
18818  18818  Money Laundering Scheme Exposed: 14 Pro-Clinto...   
18819  18819  Immigrants Head to Washington to Rally While O...   
18820  18820                      Syria Is Another Pipeline War   
18821  18821                      Voting is the Problem. Here’s   

                    author                                               text  \
0            Darrell Lucus  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          Daniel J. Flynn  Ever 

Stremming:
Stremming is the process of reducing a word to its Root word

example:
actor, actress , acting --> act

In [215]:
port_stem = PorterStemmer()


In [216]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content


In [217]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [218]:
print(news_dataset['content'])

0                darrel lucu titl
1             daniel j flynn titl
2          consortiumnew com titl
3            jessica purkiss titl
4             howard portnoy titl
                   ...           
18817    breitbart jerusalem titl
18818                        titl
18819             liz robbin titl
18820                        titl
18821                        titl
Name: content, Length: 18822, dtype: object


In [219]:
# separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [220]:
print(X)

['darrel lucu titl' 'daniel j flynn titl' 'consortiumnew com titl' ...
 'liz robbin titl' 'titl' 'titl']


In [221]:
print(Y)

[1.0 0.0 1.0 ... 0.0 1.0 '']


In [222]:
Y.shape

(18822,)

In [223]:
#converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)


In [224]:
print(X)

  (0, 3570)	0.082514618167839
  (0, 2144)	0.7046954440709177
  (0, 820)	0.7046954440709177
  (1, 3570)	0.1040133565143117
  (1, 1196)	0.8271623741190197
  (1, 805)	0.5522532286084108
  (2, 3570)	0.1176325315475847
  (2, 721)	0.8176654333075171
  (2, 702)	0.5635475372102486
  (3, 3570)	0.07333575678113471
  (3, 2849)	0.7443350230542027
  (3, 1753)	0.6637674594556724
  (4, 3570)	0.09966059503304432
  (4, 2794)	0.7081049903485587
  (4, 1578)	0.6990386888013618
  (5, 3570)	0.1219338271837369
  (5, 2582)	0.752332441773745
  (5, 805)	0.6474009876753981
  (6, 3570)	1.0
  (7, 3570)	0.09024485873911034
  (7, 3061)	0.6955478627441658
  (7, 88)	0.7127896155971838
  (8, 3570)	1.0
  (9, 3634)	0.5485815582437353
  (9, 3570)	0.06423483817477975
  :	:
  (18811, 2569)	0.704884015736682
  (18812, 3621)	0.8299766968929051
  (18812, 3570)	0.08988855418226478
  (18812, 114)	0.5505077024363645
  (18813, 3819)	0.4299209338945552
  (18813, 3570)	0.07258678715388722
  (18813, 3564)	0.6066751520733858
  (18813,

Splitting the dataset to training & test data

In [226]:
X_train, X_test , Y_train , Y_test = train_test_split(X,Y, test_size = 0.2, stratify = Y, random_state = 2)

Training the Model: Logistic Regression


In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train , Y_train)

LogisticRegression()

Evalution

accuracy score

In [ ]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction , Y_train)

In [ ]:
print('Accuracy score of the training data : ',training_data_accuracy )

Accuracy score of the training data :  0.9788461538461538


In [ ]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction , Y_test)

In [ ]:
print('Accuracy score of the test data : ',test_data_accuracy )

Accuracy score of the test data :  0.9591346153846154


Making a Predictive System

In [ ]:
X_new = X_test[3]

prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):
  print('The news is Real')

else:
  print('The news is Fake')


[0]
The news is Real


In [ ]:
print(Y_test[3])

0
